# Run Sandwich Attacks on Swaps from the Public Mempool

In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import os
import sys

current_path = sys.path[0]
sys.path.append(
    current_path[: current_path.find("defi-measurement")]
    + "liquidity-distribution-history"
)

sys.path.append("..")


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Set display options
pd.set_option('display.max_colwidth', None)  # Display entire cell content
pd.set_option('display.max_rows', 50)    # Display all rows
pd.set_option('display.max_columns', None) # Display all columns


import psycopg2
import sqlalchemy
from dotenv import load_dotenv
from matplotlib.ticker import MaxNLocator
from pool_state import v3Pool
from sqlalchemy import create_engine
from tqdm import tqdm
from collections import deque

load_dotenv(override=True)
from experiments.random_permutations import load_pool

from experiments.preload_pool_cache import load_pool_from_blob


from decimal import getcontext

getcontext().prec = 100  # Set the precision high enough for our purposes


# Read in the environment variables
postgres_uri_mp = os.environ["POSTGRESQL_URI_MP"]
postgres_uri_us = os.environ["POSTGRESQL_URI_US"]
azure_storage_uri = os.environ["AZURE_STORAGE_CONNECTION_STRING"]


## Get the Data

In [3]:
engine = create_engine(postgres_uri_mp)

query = """
SELECT *
FROM SWAP_LIMIT_PRICE AS LIM
INNER JOIN MEMPOOL_TRANSACTIONS AS MEM ON LIM.transaction_hash = MEM.HASH
"""

# col_rename = dict(
#     call_block_number='block_number',
#     contract_address='pool',
# )

# df = pd.read_sql_query(query, engine).rename(columns=col_rename).sort_values(by=['pool', 'block_number'])
df = pd.read_sql_query(query, engine)

df

,transaction_type,transaction_hash,recipient,amountIn,amountOut,amountOutMin,amountInMax,payerIsUser,token0,fee,token1,pool,hash,first_seen
0,V3_SWAP_EXACT_IN,0xf994fffef07171ffd55740b211588979d8745a964f24b1c137fa2cd5874536a2,0x0000000000000000000000000000000000000001,2300000000000000000,None,20720352176690778061354,None,false,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0x046eee2cc3188071c02bfc1745a6b17c656e3f3d,0x510100d5143e011db24e2aa38abe85d73d5b2177,0xf994fffef07171ffd55740b211588979d8745a964f24b1c137fa2cd5874536a2,2023-08-11 18:39:57.688000+00:00
1,V3_SWAP_EXACT_IN,0x68e616c69d7fc23d8669221616130baee931bb438d4aa8a686d88d390cda5323,0x0000000000000000000000000000000000000001,220000000000000000,None,1939764922511645684352,None,false,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0x046eee2cc3188071c02bfc1745a6b17c656e3f3d,0x510100d5143e011db24e2aa38abe85d73d5b2177,0x68e616c69d7fc23d8669221616130baee931bb438d4aa8a686d88d390cda5323,2023-08-11 18:36:06.730000+00:00
2,V3_SWAP_EXACT_IN,0x4f5aa8d66ab6404c628dd498979489cb173df77ed8078edf736e183d41e82dae,0x0000000000000000000000000000000000000001,300000000000000000,None,2686021351206594973990,None,false,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0x046eee2cc3188071c02bfc1745a6b17c656e3f3d,0x510100d5143e011db24e2aa38abe85d73d5b2177,0x4f5aa8d66ab6404c628dd498979489cb173df77ed8078edf736e183d41e82dae,2023-08-11 18:35:33.257000+00:00
3,V3_SWAP_EXACT_IN,0xbdbebc9d69e98aef757fc1c24ea1cf9f513f9aba3484749f070124fabf674256,0x0000000000000000000000000000000000000002,2500000000000000000000,None,270568229649369198,None,true,0x046eee2cc3188071c02bfc1745a6b17c656e3f3d,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x510100d5143e011db24e2aa38abe85d73d5b2177,0xbdbebc9d69e98aef757fc1c24ea1cf9f513f9aba3484749f070124fabf674256,2023-08-11 18:30:36.115000+00:00
4,V3_SWAP_EXACT_IN,0x7b78d768aa6cfe6f56b1fbbee88190ebe2916a37d6b9044476945d773fa24e89,0x0000000000000000000000000000000000000001,110000000000000000,None,973957923669322643285,None,false,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0x046eee2cc3188071c02bfc1745a6b17c656e3f3d,0x510100d5143e011db24e2aa38abe85d73d5b2177,0x7b78d768aa6cfe6f56b1fbbee88190ebe2916a37d6b9044476945d773fa24e89,2023-08-11 18:29:20.280000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34304,V3_SWAP_EXACT_IN,0x343a3389ff1ad1c8c8238fcac99af6fd007425883d0ddfbf6cb2d1245edaf581,0x0000000000000000000000000000000000000001,498672909100268854133,None,146490009,None,true,0x2a3bff78b79a009976eea096a51a948a3dc00e34,3000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xf4bffd826c6cf401221afb97f47c1db60d943db8,0x343a3389ff1ad1c8c8238fcac99af6fd007425883d0ddfbf6cb2d1245edaf581,2023-08-05 02:21:15.328000+00:00
34305,V3_SWAP_EXACT_IN,0x134a1c0ee347e0e12f9b6a080f9a82018622eed2c4b682e717ce05bb22e81bca,0x0000000000000000000000000000000000000001,475350906996466174,None,471811332377171658,None,true,0x20bc832ca081b91433ff6c17f85701b6e92486c5,500,0xfe2e637202056d30016725477c5da089ab0a043a,0xa9ffb27d36901f87f1d0f20773f7072e38c5bfba,0x134a1c0ee347e0e12f9b6a080f9a82018622eed2c4b682e717ce05bb22e81bca,2023-08-05 01:58:42.850000+00:00
34306,V3_SWAP_EXACT_IN,0x80804037782b7b44c2ce57bb6f0e888a056e5b36abc54e345d3745a8ffff4ccf,0x0000000000000000000000000000000000000001,21000000000000000000,None,130661221,None,false,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x80804037782b7b44c2ce57bb6f0e888a056e5b36abc54e345d3745a8ffff4ccf,2023-08-05 01:56:15.977000+00:00
34307,V3_SWAP_EXACT_IN,0xeb7a49a9921b543fcb913e18ae01edc5224ded3928483be60c840146c645c0af,0x0000000000000000000000000000000000000002,2524582283050016063314,None,103133931153910731,None,true,0x3506424f91fd33084466f402d5d97f05f8e3b4af,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x325365ed8275f6a74cac98917b7f6face8da533b,0xeb7a49a9921b543fcb913e18ae01edc5224ded3928483be60c840146c645c0af,2023-08-05 00:53:48.551000+00:00


### Populate the swap data we have with the block number that the swap appeared in

In [4]:
engine = create_engine(postgres_uri_us)

block_numbers = pd.read_sql_query(
    """
    SELECT block_number, tx_hash, block_ts
    FROM swaps
    WHERE block_number >= 17400000
    ORDER BY block_ts ASC
    """,
    engine
).set_index('tx_hash')

block_numbers

,block_number,block_ts
tx_hash,,
0xafc7a3ea2f36db267f26615eb2b94308c3225f819986073cfee134aff16692a2,17400000,2023-06-03 11:56:35
0x975564e0b0cf3595616486b015bb7e755bae1bc403204dfbda6cda8bfe0b2cce,17400000,2023-06-03 11:56:35
0x9aed05c2942cdba44b864cd5384fc0f97d93311857f4fb7b9407c60958fbf04a,17400000,2023-06-03 11:56:35
0x5ab1506abd1525fe09bb7243c0bfa3fc9a40a16fb3bfeb3ecb346863d3c5b887,17400000,2023-06-03 11:56:35
0xd833f22ec0ec84a13255b3f400809177535c2412af161386d937a02d5160eb1a,17400001,2023-06-03 11:56:47
...,...,...
0xbece543508c1e11c51b3bd2dfaae0b1e196ff5151bbc49b794bd293126a3bd34,17864000,2023-08-07 15:39:23
0x5db7b2f10a6aae5f3ba7943c59f005e1f8757004dc73b77d9585d50d8d273339,17864000,2023-08-07 15:39:23
0xe9f87658524a4529c06e9157822725469b36157624b74012716d6f2144b83762,17864005,2023-08-07 15:40:23


In [5]:
block_number_dict = block_numbers[~block_numbers.index.duplicated(keep='first')].to_dict(orient="index")

In [7]:
dataset = df.assign(block_number=df.transaction_hash.map(lambda x: block_number_dict[x]['block_number'] if x in block_number_dict else None))
dataset = dataset[~dataset.block_number.isna()]

dataset


,transaction_type,transaction_hash,recipient,amountIn,amountOut,amountOutMin,amountInMax,payerIsUser,token0,fee,token1,pool,hash,first_seen,block_number
16619,V3_SWAP_EXACT_IN,0xa0a5afcd657dab417f6004441f047b081c9ea47d4d5c5261e5c70fd96227e86c,0x0000000000000000000000000000000000000002,27500000,None,14279146475339860,None,true,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xa0a5afcd657dab417f6004441f047b081c9ea47d4d5c5261e5c70fd96227e86c,2023-08-05 14:55:51.071000+00:00,17849502.0
16631,V3_SWAP_EXACT_IN,0xbd4ed3c7f0cbe37beaab4af01f621cec8e8472850bfd1a6444d3e83263efa299,0x0000000000000000000000000000000000000002,302990056,None,163158812849422667,None,true,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xbd4ed3c7f0cbe37beaab4af01f621cec8e8472850bfd1a6444d3e83263efa299,2023-08-05 14:30:15.858000+00:00,17849374.0
16649,V3_SWAP_EXACT_IN,0x0815280508f43c94e4a0a2991f24108124effac762c120022b9e66febbe88696,0x0000000000000000000000000000000000000002,5273000000,None,2882045483002369817,None,true,0xdac17f958d2ee523a2206206994597c13d831ec7,500,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x11b815efb8f581194ae79006d24e0d814b7697f6,0x0815280508f43c94e4a0a2991f24108124effac762c120022b9e66febbe88696,2023-08-07 15:42:20.173000+00:00,17864015.0
16653,V3_SWAP_EXACT_IN,0xd6b7f04b0a02f7542805b954985ea9020dcfd98972a840081f4ee29b8506d9b7,0x0000000000000000000000000000000000000001,10000000000000000000,None,18155266696,None,false,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,500,0xdac17f958d2ee523a2206206994597c13d831ec7,0x11b815efb8f581194ae79006d24e0d814b7697f6,0xd6b7f04b0a02f7542805b954985ea9020dcfd98972a840081f4ee29b8506d9b7,2023-08-07 15:38:29.967000+00:00,17863996.0
16664,V3_SWAP_EXACT_IN,0xec263f9c66d81cb1970f671a1c6d907d3349f59a6de8d5c1454356f3997ece05,0x0000000000000000000000000000000000000001,300000000000000000,None,537970208,None,false,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,500,0xdac17f958d2ee523a2206206994597c13d831ec7,0x11b815efb8f581194ae79006d24e0d814b7697f6,0xec263f9c66d81cb1970f671a1c6d907d3349f59a6de8d5c1454356f3997ece05,2023-08-07 15:11:52.821000+00:00,17863865.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34254,V3_SWAP_EXACT_IN,0xe37078bcf2dc0786a85a74d9932d4cb32e072e415bcba918dcd7fe0b8815ec08,0x0000000000000000000000000000000000000001,70000000000000,None,1179619575207962334,None,false,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000,0xc944e90c64b2c07662a292be6244bdf05cda44a7,0x46add4b3f80672989b9a1eaf62cad5206f5e2164,0xe37078bcf2dc0786a85a74d9932d4cb32e072e415bcba918dcd7fe0b8815ec08,2023-08-05 08:21:32.150000+00:00,17847544.0
34267,V3_SWAP_EXACT_IN,0x3bcb9fe41868c827f338bfe6cce093c3e4b7f7c5b25fb986e1f9bd426d69010c,0x0000000000000000000000000000000000000001,31459982815039388609469,None,88173105,None,true,0x88303fed02b31db9c7a9eafb711da9ef4a03e5d3,3000,0xdac17f958d2ee523a2206206994597c13d831ec7,0xac5d39a35295392eada009e84ae0889d5b69bed0,0x3bcb9fe41868c827f338bfe6cce093c3e4b7f7c5b25fb986e1f9bd426d69010c,2023-08-05 08:11:58.399000+00:00,17847496.0
34280,V3_SWAP_EXACT_IN,0x0236f2774357c8ec52e086e6516476bc5d0decaf73eaa64d3a9aa0bdef084bdb,0x0000000000000000000000000000000000000002,107230444337841327371496,None,44726990681162797,None,true,0x7f3edcdd180dbe4819bd98fee8929b5cedb3adeb,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa83326d20b7003bcecf1f4684a2fbb56161e2a8e,0x0236f2774357c8ec52e086e6516476bc5d0decaf73eaa64d3a9aa0bdef084bdb,2023-08-05 06:46:59.090000+00:00,17847076.0
34281,V3_SWAP_EXACT_IN,0xb3f0775718280ce98d348ce325265c37f76a95efe9a1929a141974c2a413d98c,0x0000000000000000000000000000000000000001,96000000000000000,None,228281960750463185299147,None,false,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000,0x7f3edcdd180dbe4819bd98fee8929b5cedb3adeb,0xa83326d20b7003bcecf1f4684a2fbb56161e2a8e,0xb3f0775718280ce98d348ce325265c37f76a95efe9a1929a

In [12]:
swap_counts = dataset.groupby(by=['pool', 'block_number'])[['transaction_hash']].count().sort_values('transaction_hash', ascending=False)

swap_counts[swap_counts == 1].transaction_hash.sum(), swap_counts[swap_counts > 1].transaction_hash.sum(), swap_counts.transaction_hash.sum()

(1115.0, 8.0, 1123)

## Create Sandwich Attacks on Single Swaps

Start with this to validate the approach.

In [16]:
single_swap_blocks = swap_counts[swap_counts == 1].sort_index()

df_single = dataset.set_index(['pool', 'block_number']).loc[single_swap_blocks.index]

# Group by level 0 and count unique values in level 1
grouped_counts = df_single.groupby(level=0).apply(lambda x: x.index.get_level_values(1).nunique())

# Sort the indices based on the counts
sorted_indices = grouped_counts.sort_values(ascending=False).index

# Reindex the DataFrame based on this sorted order
df_single_sorted = df_single.loc[sorted_indices]


df_single_sorted

transaction_type  \
pool                                       block_number                     
0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 17844881.0    V3_SWAP_EXACT_IN   
                                           17844904.0    V3_SWAP_EXACT_IN   
                                           17845667.0    V3_SWAP_EXACT_IN   
                                           17845763.0    V3_SWAP_EXACT_IN   
                                           17845997.0    V3_SWAP_EXACT_IN   
...                                                                   ...   
0x824a30f2984f9013f2c8d0a29c0a3cc5fd5c0673 17851468.0    V3_SWAP_EXACT_IN   
0x2519042aa735edb4688a8376d69d4bb69431206c 17860223.0    V3_SWAP_EXACT_IN   
0x84383fb05f610222430f69727aa638f8fdbf5cc1 17860950.0    V3_SWAP_EXACT_IN   
0x8592064903ef23d34e4d5aaaed40abf6d96af186 17861372.0    V3_SWAP_EXACT_IN   
0x0188c28ea32d7860334be1a8068cfbadce18f1d9 17853849.0    V3_SWAP_EXACT_IN   

                                                                                                           transaction_hash  \
pool                                       block_number                                                                       
0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 17844881.0    0x9b725af94aeee29b275de52f7c24daa0eae7e5307005c658c5f34899035d89ee   
                                           17844904.0    0x14b5685befba09b42d2fe0f6cc7c2b0c3fd0764355f2e1fe908dd132aa324e4d   
                                           17845667.0    0xe2a288a88564e946aa9bf295d2356d3845fc48d42732710f91c2449f81cd3798   
                                           17845763.0    0xd8b8bbc709e03d77db68563fd16abd9caaf3b8d851a4c6e50ac597582788b9e2   
                                           17845997.0    0xb993834ce53cc8cec95ef92af13a3ac04dedf4cd9c954bc0ce708e78ca98f7ad   
...                                                                                                                     ...   
0x824a30f2984f9013f2c8d0a29c0a3cc5fd5c0673 17851468.0    0x470622c876dc89d5fa0dfde9cf9fe6633ceaa3085393527c1ab8d2dcbeed3c1d   
0x2519042aa735edb4688a8376d69d4bb69431206c 17860223.0    0xa79792dbe256b736e0e7bee05c5396a8f35914fc9591f3f64ee580ebaaf9cdf8   
0x84383fb05f610222430f69727aa638f8fdbf5cc1 17860950.0    0x7315d085c6e0eabe2fd192d778ad55dc84dbd436eb6d8ca919a57a3f0f619777   
0x8592064903ef23d34e4d5aaaed40abf6d96af186 17861372.0    0x99569ef3912b419a6e779951d696f82f714f7f6469e2a4613822580739f37f37   
0x0188c28ea32d7860334be1a8068cfbadce18f1d9 17853849.0    0x4f7dea75e57c80506fd03dfb35edfce4ed1a67a773e84cff7e515bd915b29053   

                                                                                          recipient  \
pool                                       block_number                                               
0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 17844881.0    0x0000000000000000000000000000000000000001   
                                           17844904.0    0x0000000000000000000000000000000000000001   
                                           17845667.0    0x0000000000000000000000000000000000000002   
                                           17845763.0    0xa8f308ab910027c5245dcfb9a5eca02444a35499   
                                           17845997.0    0x0000000000000000000000000000000000000001   
...                                                                                             ...   
0x824a30f2984f9013f2c8d0a29c0a3cc5fd5c0673 17851468.0    0x0000000000000000000000000000000000000002   
0x2519042aa735edb4688a8376d69d4bb69431206c 17860223.0    0x0000000000000000000000000000000000000002   
0x84383fb05f610222430f69727aa638f8fdbf5cc1 17860950.0    0x0000000000000000000000000000000000000001   
0x8592064903ef23d34e4d5aaaed40abf6d96af186 17861372.0    0x0000000000000000000000000000000000000001   
0x0188c28ea32d7860334be1a8068cfbadce18f1d9 17853849.0    0x0000000000000000000000000000000000000001   

                                                               

In [18]:
# Keep only the swap in subset for now
df_single_sorted = df_single_sorted[df_single_sorted.transaction_type == 'V3_SWAP_EXACT_IN'].drop(columns=['transaction_type', 'recipient', 'amountOut', 'amountInMax', 'payerIsUser', 'transaction_hash'])

df_single_sorted

amountIn  \
pool                                       block_number                                                                                  
0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 17844881.0                                                               820000000000000000   
                                           17844904.0                                                             13000000000000000000   
                                           17845667.0                                                                         18325893   
                                           17845763.0    57896044618658097711785492504343953926634992332820282019728792003956564819968   
                                           17845997.0                                                                 3000000000000000   
...                                                                                                                                ...   
0x824a30f2984f9013f2c8d0a29c0a3cc5fd5c0673 17851468.0                                                             59594801831491290000   
0x2519042aa735edb4688a8376d69d4bb69431206c 17860223.0                                                              2084378349167516977   
0x84383fb05f610222430f69727aa638f8fdbf5cc1 17860950.0                                                                        217031721   
0x8592064903ef23d34e4d5aaaed40abf6d96af186 17861372.0                                                           6632804553767876598518   
0x0188c28ea32d7860334be1a8068cfbadce18f1d9 17853849.0                                                               200000000000000000   

                                                                  amountOutMin  \
pool                                       block_number                          
0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 17844881.0               1488759839   
                                           17844904.0              23596354509   
                                           17845667.0         9554841306778577   
                                           17845763.0                621700347   
                                           17845997.0                  5219922   
...                                                                        ...   
0x824a30f2984f9013f2c8d0a29c0a3cc5fd5c0673 17851468.0         9057065915305617   
0x2519042aa735edb4688a8376d69d4bb69431206c 17860223.0         4369482744300305   
0x84383fb05f610222430f69727aa638f8fdbf5cc1 17860950.0    121685518570314610959   
0x8592064903ef23d34e4d5aaaed40abf6d96af186 17861372.0               4154907937   
0x0188c28ea32d7860334be1a8068cfbadce18f1d9 17853849.0         1480449673819242   

                                                                                             token0  \
pool                                       block_number                                               
0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 17844881.0    0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
                                           17844904.0    0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
                                           17845667.0    0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48   
                                           17845763.0    0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
                                           17845997.0    0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
...                                                                                             ...   
0x824a30f2984f9013f2c8d0a29c0a3cc5fd5c0673 17851468.0    0x5283d291dbcf85356a21ba090e6db59121208b44   
0x2519042aa735edb4688a8376d69d4bb69431206c 17860223.0    0x58b6a8a3302369daec383334672404ee733ab239   
0x84383fb05f610222430f69727aa638f8fdbf5cc1 17860950.0    0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48   
0x8592064903ef23d34e4d5aaaed40abf6d96af186 17861372.0    0xaf5191b0de278c7286d6c7cc6ab6bb8a73ba2cd6   
0x0188c28ea32d7860334be1a806

## Get a pool and do some initial testing

In [24]:
pool = load_pool_from_blob(
    "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
    postgres_uri_us,
    azure_storage_uri,
    "uniswap-v3-pool-cache",
    verbose=True,
)

Loading pool from Azure blob storage cache


In [ ]:
assert pool is not None

for (pool_addr, block), swap in df_single_sorted.iterrows(): # type: ignore


    print(f"Pool: {pool_addr}, Block: {block}")

    

    swap_params =         {
        "input": int(swap.amountIn),
        "tokenIn": swap.token0,
        "as_of": block,
        "gasFee": True,
    }

    output, heur = pool.swap_in(swap_params)

    print(f"Swap ({swap.amountSpecified} for {output})")
    print(f"Next sqrt price: {heur.sqrtP_next} vs. sqrtPriceLimit: {swap.sqrtPriceLimitX96}")
    print(f"Sqrt Price: {heur.sqrt_P}")

    assert False


Pool: call_success
Block: 1.0


IndexError: single positional indexer is out-of-bounds

## Run Sandwich attacks on multiple swaps

In [7]:
def get_mev_single():
    pass

In [ ]:
swap

,pool,call_success,call_tx_hash,call_trace_address,call_block_time,block_number,amountSpecified,output_amount0,output_amount1,recipient,sqrtPriceLimitX96,zeroForOne,hash,first_seen
25074,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,True,0xa1dc1bc7b54c6a13c48718714cf9524f34621d955b61...,[0],2023-06-24 22:01:59+00:00,17552211,5000000000,5000000000,-2663464990371447655,0xb780f89d37864492a81c7b0053879486d259f16c,4295128740,True,0xa1dc1bc7b54c6a13c48718714cf9524f34621d955b61...,2023-06-24 22:01:48.527000+00:00
98402,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,True,0xa5a2a94332d6c29d167f6bfac54b541ac056203b62b7...,[0 3 3 0 0],2023-06-24 22:01:59+00:00,17552211,452238284,452238284,-240903021710830509,0xdef1c0ded9bec7f1a1670819833240f027b25eff,4295128740,True,0xa5a2a94332d6c29d167f6bfac54b541ac056203b62b7...,2023-06-24 22:01:50.697000+00:00
575369,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,True,0xcccbb87fc3e5e2b7851adc2140f46603c39eebef6480...,[0],2023-06-24 22:01:59+00:00,17552211,3200000000,3200000000,-1704604062672965831,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,4295128740,True,0xcccbb87fc3e5e2b7851adc2140f46603c39eebef6480...,2023-06-24 22:01:55.725000+00:00


In [ ]:
from decimal import Decimal, getcontext

# Ensure that we have enough precision
getcontext().prec = 100

# Example sqrtPriceLimitX96 value
sqrt_price_limit_x96_str = "4295128740"
sqrt_price_limit_x96 = Decimal(sqrt_price_limit_x96_str)

# Convert to the square root of the price
sqrt_price = sqrt_price_limit_x96 / (2**96)

# Calculate the actual price
actual_price = sqrt_price * sqrt_price

sqrt_price, actual_price


(Decimal('5.42121463340349417168512463012009817975300762782353558577597141265869140625E-20'),
 Decimal('2.938956810142818170490203137305544242974071560518843705584884307271284930443817505824327569078380388E-39'))